# Tutorial 8: Obtaining Realistic Cherenkov Events

While it is possible to create toy Cherenkov events with `PhotoelectronSource.get_cherenkov_shower`, there are no limitations on the parameters supplied. The resulting Cherenkov showers can consequently be very unrealistic. Additionally, there is no underlying relation of the parameters to the typical physical energy spectra. To reliably describe the on-sky performance of a camera simulated with sstcam-simulation, we must be able to simulate its response to realistic Cherenkov showers.

This tutorial demonstrates the `SimtelReader` class, reading photoelectrons saved inside simtelarray files, which in addition to the arrival pixel, time and charge of the photoelectrons, provides metadata about the simulated Cherenkov shower event (e.g. energy). It also demonstrates processing these photoelectrons through the sstcam-simulation chain, resampling the charge with the spectrum we define, adding NSand generating the waveform

In [6]:
from sstcam_simulation import Camera, SimtelReader, EventAcquisition, PhotoelectronSource
from sstcam_simulation.camera.pulse import GaussianPulse
from sstcam_simulation.plotting import CameraImage
from sstcam_simulation.data import get_data
from matplotlib import pyplot as plt
%matplotlib

Using matplotlib backend: MacOSX


In [7]:
camera = Camera(
    continuous_readout_duration=128,
)

In [11]:
# Obtain test dataset
path = get_data("testing/simtel_test.simtel.gz")

In [12]:
SimtelReader?

Init signature:
SimtelReader(
    path: str,
    disable_remapping: bool = False,
    only_triggered_events: bool = False,
    n_events: int = None,
)
Docstring:      <no docstring>
Init docstring:
Read Photoelectron arrays directly from simtelarray files

Parameters
----------
path : str
    Path to the simtel file
disable_remapping : bool
    Disables the remapping of the pixels to the sstcam-simulation
    pixel mapping
only_triggered_events : bool
    Only read events which caused a telescope trigger
n_events : int
    Number of telescope events to process
File:           ~/Software/sstcam-simulation/sstcam_simulation/io/simtel_reader.py
Type:           type
Subclasses:     


In [21]:
# Obtain first Cherenkov event in file
reader = SimtelReader(path)
for pe in reader:
    if len(pe) > 200:
        break
        
# Resample the charge using the spectrum defined in the Camera
acquisition = EventAcquisition(camera=camera)
pe = acquisition.resample_photoelectron_charge(pe)

# Simulate some nsb
source = PhotoelectronSource(camera=camera)
nsb = source.get_nsb(10)
readout = acquisition.get_continuous_readout(pe+nsb)
samples = acquisition.get_sampled_waveform(readout)

In [22]:
image = CameraImage.from_coordinates(camera.mapping.pixel)
image.add_colorbar("Total Number of Photoelectrons")
image.image = pe.get_charge_per_pixel(camera.mapping.n_pixels)

In [16]:
import time
image = CameraImage.from_coordinates(camera.mapping.pixel)
image.add_colorbar("Waveform Amplitude")
image.set_limits_minmax(samples.min(), samples.max())
n_samples = samples.shape[1]
#plt.pause(10)
for isample in range(n_samples):
    image.image = samples[:, isample]
    plt.pause(0.01)

In [18]:
_ = plt.plot(samples.T)